In [1]:
import mysql.connector
import pandas as pd

In [5]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()

In [12]:
# Create stored procedure: comput total order amount
cursor.execute("DROP PROCEDURE IF EXISTS ComputeTotalOrderAmount")
cursor.execute("""
CREATE PROCEDURE ComputeTotalOrderAmount(IN customer_id INT, OUT total_order_amount INT)
BEGIN
    SELECT COUNT(o.id) INTO total_order_amount
    FROM orders o
    WHERE o.customer_id = customer_id;
END;
""")
connection.commit()
cursor.close()
connection.close()
print("Stored procedure created.")


Stored procedure created.


In [18]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()
args = [1, 0]  # 0 là placeholder cho OUT
results = cursor.callproc('ComputeTotalOrderAmount', args)

# OUT nằm ở vị trí thứ 2 trong mảng args
print("Tổng số đơn hàng:", results[1])

Tổng số đơn hàng: 2


In [19]:
#Tạo procedure get name of employee by id
cursor.execute("DROP PROCEDURE IF EXISTS GetEmployeeNameById")
cursor.execute("""
CREATE PROCEDURE GetEmployeeNameById(IN employee_id INT, OUT employee_name VARCHAR(255))
BEGIN
    SELECT name INTO employee_name
    FROM employees
    WHERE id = employee_id;
END;
""")
connection.commit()
cursor.close()
connection.close()
print("Stored procedure created.")

Stored procedure created.


In [21]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()
args = [2, 0]  # 0 là placeholder cho OUT
results = cursor.callproc('GetEmployeeNameById', args)

# OUT nằm ở vị trí thứ 2 trong mảng args
print("Tên nhân viên:", results[1])

Tên nhân viên: Emma


In [22]:
# Update customer city
cursor.execute("DROP PROCEDURE IF EXISTS UpdateCustomerCity")
cursor.execute("""
CREATE PROCEDURE UpdateCustomerCity(IN customer_id INT, IN new_city VARCHAR(255))
BEGIN
    UPDATE customers
    SET city = new_city
    where id = customer_id;
END;
""")
connection.commit()
cursor.close()
connection.close()
print("Stored procedure created.")


Stored procedure created.


In [24]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)

cursor = connection.cursor()
args = [2, 'New York']  # 0 là placeholder cho OUT
results = cursor.callproc('UpdateCustomerCity', args)

# OUT nằm ở vị trí thứ 2 trong mảng args
print("Tên nhân viên:", results[1])

df = pd.read_sql_query("SELECT * FROM customers", connection)
print(df)


Tên nhân viên: New York
   id     name      city
0   1    Alice     HaNoi
1   2      Bob  New York
2   3  Charlie     Hanoi


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_8916\481369384.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM customers", connection)


In [31]:
#Print product haveing price > X
cursor.execute("DROP PROCEDURE IF EXISTS PrintProductHavingPriceGreaterThanX")
cursor.execute("""
CREATE PROCEDURE PrintProductHavingPriceGreaterThanX(IN min_price DECIMAL(10,2))
BEGIN
    SELECT * 
    FROM products 
    WHERE price > min_price;
END;
""")
connection.commit()
cursor.close()
connection.close()
print("Stored procedure created.")



Stored procedure created.


In [ ]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)
cursor = connection.cursor()

min_price = 70.00
results = cursor.callproc('PrintProductHavingPriceGreaterThanX', [min_price])

# Lấy kết quả từ stored procedure
for result in cursor.stored_results():
    rows = result.fetchall()
    for row in rows:
        print(row)

(1, 'Laptop', Decimal('1500.00'))
(3, 'Keyboard', Decimal('100.00'))
(4, 'Monitor', Decimal('300.00'))


In [44]:
#Delete order have total amount < X
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)
cursor = connection.cursor()

cursor.execute("DROP PROCEDURE IF EXISTS DeleteOrderHavingTotalAmountLessThanX")
cursor.execute("""
CREATE PROCEDURE DeleteOrderHavingTotalAmountLessThanX(IN min_total DECIMAL(10,2))
BEGIN
    SELECT * FROM orders WHERE total < min_total;
               
    DELETE FROM order_assignments 
    WHERE order_id IN 
               (SELECT id FROM orders WHERE total < min_total);

    DELETE FROM orders
    WHERE total < min_total;
END;
""")
connection.commit()
cursor.close()
connection.close()
print("Stored procedure created.")


Stored procedure created.


In [48]:
#Connect to database
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="123#edc",
    database="aio.week.10"
)
cursor = connection.cursor()

cursor.callproc('DeleteOrderHavingTotalAmountLessThanX', [500])

for result in cursor.stored_results():
    rows = result.fetchall()
    print("Đơn hàng sẽ bị xóa:")
    for row in rows:
        print(row)

connection.commit()

Đơn hàng sẽ bị xóa:


In [49]:
query = pd.read_sql_query("SELECT * FROM orders", connection)
print(query)

   id  customer_id order_date    total
0   1            1 2024-01-10    500.0
1   2            1 2024-03-01    800.0
2   3            2 2024-02-20  12000.0


C:\Users\AI LEARNER\AppData\Local\Temp\ipykernel_8916\4075030552.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query = pd.read_sql_query("SELECT * FROM orders", connection)
